# Stabilizer Control Notebook

Conglomeration of code to run the sinara stabilizer. Comments and code heavily influenced by chatGPT

### Install Dependencies

In [45]:
# -----------------------------------------------------------------------------
# Install Packages
# -----------------------------------------------------------------------------
#Note: minconf and paho.mqtt may need to be installed, use "uv pip install miniconf paho.mqtt"



import miniconf
from miniconf import Miniconf

import paho.mqtt.client as mqtt

import stabilizer_filter_design
from stabilizer_filter_design import (
    FilterLibrary,
    StabilizerParameters,
)

import asyncio

# --- Module locations ---
print("miniconf module file:               ", miniconf.__file__)
print("paho.mqtt.client module file:       ", mqtt.__file__)
print("stabilizer_filter_design module file:", stabilizer_filter_design.__file__)
print("asyncio module file:                ", asyncio.__file__)

# --- Class origins (sanity check) ---
print("Miniconf class defined in module:           ", Miniconf.__module__)
print("FilterLibrary class defined in module:      ", FilterLibrary.__module__)
print("StabilizerParameters class defined in module:", StabilizerParameters.__module__)




miniconf module file:                C:\Users\scientist\code\juno\.venv\Lib\site-packages\miniconf\__init__.py
paho.mqtt.client module file:        C:\Users\scientist\code\juno\.venv\Lib\site-packages\paho\mqtt\client.py
stabilizer_filter_design module file: C:\Users\scientist\code\jam\ronan\sinara_stabilizer_revamp\stabilizer_filter_design.py
asyncio module file:                 C:\Users\scientist\AppData\Roaming\uv\python\cpython-3.11.13-windows-x86_64-none\Lib\asyncio\__init__.py
Miniconf class defined in module:            miniconf.miniconf
FilterLibrary class defined in module:       stabilizer_filter_design
StabilizerParameters class defined in module: stabilizer_filter_design


### Raw IIR coefficient upload (Sinara stabilizer)

In [272]:
# -----------------------------------------------------------------------------
# Raw biquad filter support (direct coefficient upload, no reinterpretation)
# -----------------------------------------------------------------------------
def make_raw_payload(ba, offset=0, lo=lowVoltageLim, hi=highVoltageLim):
    """
    Construct the full payload required by the Sinara 'Raw' biquad filter.

    Parameters
    ----------
    ba : iterable
        The raw biquad coefficients in DSP order.
        Typically [b0, b1, b2, a1, a2].
        These should already be scaled appropriately for the FPGA.
    offset : int, optional
        Constant offset added to the filter output (DAC units).
        Defaults to 0.
    lo : int, optional
        Minimum clamp value for the filter output.
        Defaults to -32767 (full-scale negative for a signed 16-bit DAC).
    hi : int, optional
        Maximum clamp value for the filter output.
        Defaults to +32767 (full-scale positive for a signed 16-bit DAC).

    Returns
    -------
    dict
        A dictionary matching the exact structure expected by
        /repr/Raw on the stabilizer.
    """
    return {
        # Convert all coefficients to floats to avoid MQTT / JSON type ambiguity
        "ba": [float(x) for x in ba],

        # Output offset applied after filtering
        "u": int(offset),

        # Output saturation limits (hardware safety + numerical stability)
        "min": int(lo),
        "max": int(hi),
    }


async def apply_filter(ba, offset=0):
    """
    Apply a Raw biquad filter to the selected channel on the stabilizer.

    This function:
      1. Connects to the stabilizer over MQTT
      2. Forces the biquad type to 'Raw'
      3. Uploads the full Raw representation payload
      4. Starts the channel processing

    Parameters
    ----------
    ba : iterable
        Raw biquad coefficients [b0, b1, b2, a1, a2].
    offset : int, optional
        Output offset (DAC units). Defaults to 0.
    """

    # Open an asynchronous MQTT connection to the broker
    async with miniconf.Client(broker, protocol=miniconf.MQTTv5) as client:
        # Bind the client to the stabilizer's configuration namespace
        dev = miniconf.Miniconf(client, prefix)

        # Explicitly set the biquad filter type to "Raw"
        # This tells the firmware NOT to reinterpret coefficients
        await dev.set(f"/ch/{channel}/biquad/0/typ", "Raw")

        # Upload the *entire* Raw biquad payload
        # This is critical: partial updates can leave stale values
        # inside the FPGA configuration.
        await dev.set(
            f"/ch/{channel}/biquad/0/repr/Raw",
            make_raw_payload(ba, offset),
        )

        # Start (or restart) the channel so the new filter takes effect
        await dev.set(f"/ch/{channel}/run", "Run")

        print(f"Filter applied for {ba}")

async def set_stream_target(computer_ip, port=9293):
    async with miniconf.Client(broker, protocol=miniconf.MQTTv5) as client:
        dev = miniconf.Miniconf(client, prefix)
        await dev.set("/stream", f"{computer_ip}:{port}")
        print(f"Streaming target set to {computer_ip}:{port}")


async def enable_pi_with_preload(ba, desired_dac):

    async with miniconf.Client(broker, protocol=miniconf.MQTTv5) as client:
        dev = miniconf.Miniconf(client, prefix)

        # 1️⃣ Freeze loop
        await dev.set(f"/ch/{channel}/run", "Hold")

        # 2️⃣ Load unity filter with preload offset
        unity_ba = [1, 0, 0, 0, 0]

        await dev.set(f"/ch/{channel}/biquad/0/typ", "Raw")
        await dev.set(
            f"/ch/{channel}/biquad/0/repr/Raw",
            make_raw_payload(unity_ba, offset=desired_dac),
        )

        # Small settle time (important!)
        await asyncio.sleep(0.05)

        # 3️⃣ Load PI filter while preserving offset
        await dev.set(
            f"/ch/{channel}/biquad/0/repr/Raw",
            make_raw_payload(ba, offset=desired_dac),
        )

        # 4️⃣ Enable loop
        await dev.set(f"/ch/{channel}/run", "Run")
        
def make_pid_payload(
    kp,
    ki,
    kd,
    setpoint,
    preload,
    lo=lowVoltageLim,
    hi=highVoltageLim,
):
    return {
        "kp": float(kp),
        "ki": float(ki),
        "kd": float(kd),

        "setpoint": float(setpoint),

        # preload integrator / controller output
        "u": float(preload),

        "min": int(lo),
        "max": int(hi),
    }


async def apply_pid(kp, ki, kd, setpoint, preload=0):

    async with miniconf.Client(broker, protocol=miniconf.MQTTv5) as client:
        dev = miniconf.Miniconf(client, prefix)

        # Stop channel first
        await dev.set(f"/ch/{channel}/run", "Hold")

        # Set PID type
        await dev.set(f"/ch/{channel}/biquad/0/typ", "Pid")

        # Upload full payload
        await dev.set(
            f"/ch/{channel}/biquad/0/repr/Pid",
            make_pid_payload(kp, ki, kd, setpoint, preload),
        )

        # Enable loop
        await dev.set(f"/ch/{channel}/run", "Run")


async def enable_pid_with_preload(
    kp,
    ki,
    kd,
    setpoint,
    desired_dac,
):

    async with miniconf.Client(broker, protocol=miniconf.MQTTv5) as client:
        dev = miniconf.Miniconf(client, prefix)

        # Freeze loop
        await dev.set(f"/ch/{channel}/run", "Hold")

        # Select PID mode
        await dev.set(f"/ch/{channel}/biquad/0/typ", "Pid")

        # --- Gains ---
        await dev.set(f"/ch/{channel}/biquad/0/repr/Pid/gain/p", float(kp))
        await dev.set(f"/ch/{channel}/biquad/0/repr/Pid/gain/i", float(ki))
        await dev.set(f"/ch/{channel}/biquad/0/repr/Pid/gain/d", float(kd))

        # Disable higher order terms
        await dev.set(f"/ch/{channel}/biquad/0/repr/Pid/gain/i2", 0.0)
        await dev.set(f"/ch/{channel}/biquad/0/repr/Pid/gain/d2", 0.0)

        # --- Limits ---
        await dev.set(f"/ch/{channel}/biquad/0/repr/Pid/min", int(lowVoltageLim))
        await dev.set(f"/ch/{channel}/biquad/0/repr/Pid/max", int(highVoltageLim))

        # --- Setpoint ---
        await dev.set(
            f"/ch/{channel}/biquad/0/repr/Pid/setpoint",
            float(setpoint),
        )

        # ⭐ Integrator preload (working version)
        await dev.set(
            f"/ch/{channel}/biquad/0/repr/Pid/limit/i",
            float(desired_dac),
        )

        await asyncio.sleep(0.05)

        print("PID state:")
        print("setpoint:", await dev.get(f"/ch/{channel}/biquad/0/repr/Pid/setpoint"))
        print("kp:", await dev.get(f"/ch/{channel}/biquad/0/repr/Pid/gain/p"))
        print("ki:", await dev.get(f"/ch/{channel}/biquad/0/repr/Pid/gain/i"))
        print("integrator:", await dev.get(f"/ch/{channel}/biquad/0/repr/Pid/limit/i"))

        # Enable loop
        await dev.set(f"/ch/{channel}/run", "Run")


async def dump_pid_tree():

    async with miniconf.Client(broker, protocol=miniconf.MQTTv5) as client:
        dev = miniconf.Miniconf(client, prefix)

        base = f"/ch/{channel}/biquad/0/repr/Pid"

        # --- Get child keys ---
        try:
            keys = await dev.get(base)
            keys = [base]   # rare case where it is leaf
        except AssertionError as e:
            # Miniconf stores returned keys inside AssertionError args
            keys = e.args[0]

        print("\nPID configuration:")
        print("-------------------")

        for key in keys:
            try:
                val = await dev.get(key)
                print(f"{key} = {val}")
            except Exception as err:
                print(f"{key} -> ERROR: {err}")


async def check_filter_type():
    async with miniconf.Client(broker, protocol=miniconf.MQTTv5) as client:
        dev = miniconf.Miniconf(client, prefix)

        typ = await dev.get(f"/ch/{channel}/biquad/0/typ")
        print("Filter type:", typ)

async def probe_pid_orders():
    """
    Check which PID controller orders are supported by the stabilizer.
    Avoids 'client not connected' by managing MQTT internally.
    """

    async with miniconf.Client(broker, protocol=miniconf.MQTTv5) as client:
        dev = miniconf.Miniconf(client, prefix)

        base = f"/ch/{channel}/biquad/0/repr/Pid/order"

        print("\nTesting PID order support")
        print("-------------------------")

        # Show current order first
        try:
            current = await dev.get(base)
            print("Current order:", current)
        except Exception as e:
            print("Could not read current order:", e)

        # Try common enum values
        test_values = ["I", "PI", "PID", 0, 1, 2]

        for val in test_values:
            try:
                await dev.set(base, val)
                confirmed = await dev.get(base)
                print(f"✅ Accepted {val} -> Device reports {confirmed}")
            except Exception as e:
                print(f"❌ Rejected {val} -> {e}")



### Initial Configuration

In [199]:
# -----------------------------------------------------------------------------
# Initial Stabilizer Config
# -----------------------------------------------------------------------------
broker = "192.168.1.222"            # replace with your broker IP or hostname
stabilizer_name = "44-b7-d0-cc-65-c0" # check this name from the MQTT explorer window
prefix = f"dt/sinara/dual-iir/{stabilizer_name}"       # topic to subscribe to
channel = 0
computer_ip = "192.168.1.134" #IP address of the computer you are running this on
lowVoltageLim = -6553 #abs minimum is -32767
highVoltageLim = 6553 #abs max is 32767
await set_stream_target(f"{computer_ip}")





Streaming target set to 192.168.1.134:9293


## Stabilizer Control 

### Documentation 

In [186]:
"""
========================= PI FILTER USAGE =========================

ba = FilterLibrary.get_ba(
    "PI",
    f0=XX,  # PI crossover frequency in Hz
    g=X.X,    # integrator strength (lower = stronger integration)
    K=X.X,    # overall controller gain
)

This call constructs a *digital* PI controller using the analytic
transfer function defined in FilterLibrary.library:

    H(s) = K * (1 + s / (2π f0)) / (1/g + s / (2π f0))

This is a *physically meaningful* PI controller expressed in the
continuous-time s-domain, then converted to a discrete-time
biquad using a bilinear (Tustin) transform.

Parameter meanings:

  f0 : Crossover / corner frequency (Hz)
       - Sets where the controller transitions from integral-dominated
         behavior (low frequency) to proportional-dominated behavior
         (high frequency).
       - Below f0: strong integral action → eliminates DC error.
       - Above f0: mostly proportional action → improves stability.

  g  : Integrator strength (dimensionless)
       - Controls the DC gain of the integrator.
       - Smaller g  → stronger integral action (higher low-frequency gain).
       - Larger g   → weaker integral action.
       - g = 1.0 corresponds to a "textbook" PI shape.

  K  : Overall loop gain (dimensionless)
       - Scales the entire controller.
       - This directly affects loop stability and bandwidth.

Internally, this function:
  1. Builds the symbolic transfer function H(s)
  2. Applies a bilinear transform:
       s → (2/Ts) * (1 - z⁻¹) / (1 + z⁻¹)
  3. Normalizes the result into Stabilizer format:
       [b0, b1, b2, -a1, -a2]

The returned coefficients `ba` are *ready to be uploaded*
directly to a Stabilizer Raw biquad.

Example (20 kHz crossover, unity integrator strength, unity gain):
==================================================================
"""


'\n========================= PI FILTER USAGE =========================\n\nba = FilterLibrary.get_ba(\n    "PI",\n    f0=XX,  # PI crossover frequency in Hz\n    g=X.X,    # integrator strength (lower = stronger integration)\n    K=X.X,    # overall controller gain\n)\n\nThis call constructs a *digital* PI controller using the analytic\ntransfer function defined in FilterLibrary.library:\n\n    H(s) = K * (1 + s / (2π f0)) / (1/g + s / (2π f0))\n\nThis is a *physically meaningful* PI controller expressed in the\ncontinuous-time s-domain, then converted to a discrete-time\nbiquad using a bilinear (Tustin) transform.\n\nParameter meanings:\n\n  f0 : Crossover / corner frequency (Hz)\n       - Sets where the controller transitions from integral-dominated\n         behavior (low frequency) to proportional-dominated behavior\n         (high frequency).\n       - Below f0: strong integral action → eliminates DC error.\n       - Above f0: mostly proportional action → improves stability.\n\n  g

In [187]:
"""
==================== APPENDIX: HOW TO TUNE THE PI IN PRACTICE ====================

This PI controller is intended to be tuned *against a real plant*.
Do NOT treat (f0, g, K) as abstract numbers — each one maps cleanly
onto a physical loop property.

Recommended tuning procedure:

------------------------------------------------------------------
STEP 0 — MEASURE OR ESTIMATE YOUR PLANT
------------------------------------------------------------------
You must have (or estimate) the plant transfer function P(f), either
from:
  • a measured Bode plot, or
  • a known physical model, or
  • a swept sine / chirp response

Key things to identify:
  • Dominant plant pole(s)
  • Where phase begins to roll off
  • Actuator and sensor limits

If you cannot estimate P(f), PI tuning will be guesswork.

------------------------------------------------------------------
STEP 1 — CHOOSE f0 (PI ZERO LOCATION)
------------------------------------------------------------------
Choose f0 *below* the dominant plant pole.

Rule of thumb:
  f0 ≈ (0.1 to 0.3) × f_pole

Why:
  • Below f0, integral action dominates → eliminates DC error
  • Above f0, proportional action dominates → avoids phase loss
  • Putting the zero too high causes instability
  • Putting it too low makes the loop sluggish

Example:
  Plant pole at ~100 kHz
  → choose f0 ≈ 10–30 kHz

------------------------------------------------------------------
STEP 2 — SET g (INTEGRATOR STRENGTH)
------------------------------------------------------------------
Start with:
  g = 1.0

Then adjust only if needed:

  Smaller g (e.g. 0.3):
    • Stronger integral action
    • Faster DC error correction
    • Higher risk of oscillation / windup

  Larger g (e.g. 3–10):
    • Weaker integrator
    • More stable, but slower convergence

Rule:
  If the loop oscillates slowly → increase g
  If DC error persists too long → decrease g

------------------------------------------------------------------
STEP 3 — TUNE K (OVERALL LOOP GAIN)
------------------------------------------------------------------
This is the *most sensitive parameter*.

Procedure:
  1. Start with K ≪ 1 (e.g. 0.01)
  2. Increase K slowly while observing the system response
  3. Stop when you see:
       • ringing
       • overshoot
       • sustained oscillation
  4. Back off K by ~30–50%

Interpretation:
  • K controls bandwidth
  • Too large → instability
  • Too small → weak control authority

------------------------------------------------------------------
STEP 4 — CHECK OUTPUT LIMITS (VERY IMPORTANT)
------------------------------------------------------------------
Ensure output limits are configured correctly:

  y_min, y_max should match actuator capability

Why:
  • Integral windup occurs if the output saturates
  • Windup looks like slow recovery or runaway behavior

If saturation is unavoidable:
  • Reduce K
  • Increase g
  • Lower f0

------------------------------------------------------------------
STEP 5 — VERIFY IN FREQUENCY DOMAIN
------------------------------------------------------------------
After tuning, verify:
  • Phase margin ≥ 45°
  • Gain margin ≥ 6 dB
  • No excessive peaking near f0

Use:
  FilterLibrary.bode("PI", ...)
  or
  bode(ba, fs)

------------------------------------------------------------------
SUMMARY OF PARAMETER ROLES
----------------
"""

'\n==================== APPENDIX: HOW TO TUNE THE PI IN PRACTICE ====================\n\nThis PI controller is intended to be tuned *against a real plant*.\nDo NOT treat (f0, g, K) as abstract numbers — each one maps cleanly\nonto a physical loop property.\n\nRecommended tuning procedure:\n\n------------------------------------------------------------------\nSTEP 0 — MEASURE OR ESTIMATE YOUR PLANT\n------------------------------------------------------------------\nYou must have (or estimate) the plant transfer function P(f), either\nfrom:\n  • a measured Bode plot, or\n  • a known physical model, or\n  • a swept sine / chirp response\n\nKey things to identify:\n  • Dominant plant pole(s)\n  • Where phase begins to roll off\n  • Actuator and sensor limits\n\nIf you cannot estimate P(f), PI tuning will be guesswork.\n\n------------------------------------------------------------------\nSTEP 1 — CHOOSE f0 (PI ZERO LOCATION)\n---------------------------------------------------------------

### Commands

#### Turn Off

In [303]:
ba = [0, 0, 0, 0, 0,] #[b0, b1, b2, -a1, -a2] 
await apply_filter(ba)


Filter applied for [0, 0, 0, 0, 0]


#### Lump Sum

In [301]:
ba = [0, 0, 0, 0, 0,] #[b0, b1, b2, -a1, -a2] 
await apply_filter(ba, 670)

Filter applied for [0, 0, 0, 0, 0]


## Tune

In [222]:
ba = FilterLibrary.get_ba("PI", f0=20, g=0.005, K=0.002)
await apply_filter(ba, 670)


Filter applied for [0.0019684976545605616, -0.001968181048073733, 0.0, 0.9683393513171474, 0.0]


#### K tuning

In [219]:
ba = FilterLibrary.get_ba("P", K=0.002)
await apply_filter(ba, 670)

Filter applied for [0.002, 0.0, 0.0, 0.0, 0.0]


#### Quick Voltage Converter


In [120]:
voltage = 0.19129528
bits= 600

print(f"{voltage} Volts is {((voltage/10)*32767):.3f} bits")
print(f"{bits} Bits is {(bits/32767)*10:.4f} Volts")


0.19129528 Volts is 626.817 bits
600 Bits is 0.1831 Volts


In [252]:
ba = FilterLibrary.get_ba("PI", f0=20, g=0.005, K=0.002)

await enable_pi_with_preload(ba, desired_dac=670)


PID state:
setpoint: 3.0
kp: -0.002
ki: 0.0
integrator: 670.0


In [276]:
await check_filter_type()
await dump_pid_tree()

Filter type: "Pid"

PID configuration:
-------------------
/ch/0/biquad/0/repr/Pid/order = "I"
/ch/0/biquad/0/repr/Pid/gain/i2 = 0.0
/ch/0/biquad/0/repr/Pid/gain/i = 0.005
/ch/0/biquad/0/repr/Pid/gain/p = 0.002
/ch/0/biquad/0/repr/Pid/gain/d = 0.1
/ch/0/biquad/0/repr/Pid/gain/d2 = 0.0
/ch/0/biquad/0/repr/Pid/limit/i2 = null
/ch/0/biquad/0/repr/Pid/limit/i = 670.0
/ch/0/biquad/0/repr/Pid/limit/p = null
/ch/0/biquad/0/repr/Pid/limit/d = null
/ch/0/biquad/0/repr/Pid/limit/d2 = null
/ch/0/biquad/0/repr/Pid/setpoint = 3.0
/ch/0/biquad/0/repr/Pid/min = -6553.0
/ch/0/biquad/0/repr/Pid/max = 6553.0


In [302]:
await enable_pid_with_preload(
    kp=0.002,
    ki=0.005,
    kd=0.0,
    setpoint=3.0,
    desired_dac=670,
)


PID state:
setpoint: 3.0
kp: 0.002
ki: 0.005
integrator: 670.0



Testing PID order support
-------------------------
Current order: "I"
✅ Accepted I -> Device reports "I"
❌ Rejected PI -> ('Error', '(De)serialization (depth 7)')
❌ Rejected PID -> ('Error', '(De)serialization (depth 7)')
❌ Rejected 0 -> ('Error', '(De)serialization (depth 7)')
❌ Rejected 1 -> ('Error', '(De)serialization (depth 7)')
❌ Rejected 2 -> ('Error', '(De)serialization (depth 7)')
